In [1]:
%gui qt
import napari
import pandas as pd
import numpy as np
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
from shapely.geometry import Polygon, Point

## Make a napari viewer...

In [3]:
v = napari.Viewer()

## Load cell x gene data from 1 dataset

In [36]:
all_cells = pd.read_csv("/Users/brian/Downloads/1001844875_2_output_dataframe.csv",skiprows=5)
position_array=np.array([[np.float(position.split(' ')[0].strip('[')), np.float(position.split(' ')[1].strip(']'))] for position in all_cells["position"]])
all_cells["x_um"] = position_array[:,0]
all_cells["y_um"] = position_array[:,1]
all_cells.to_csv("/Users/brian/Downloads/fixed_1001844875.csv")

In [35]:
all_cells.head(10)

,Unnamed: 0,1700022I11Rik,1810046K07Rik,5031425F14Rik,5730522E02Rik,Acta2,Adam2,Adamts2,Adamts4,Adra1b,...,Vwc2,Wipf3,Wnt7b,Zfp804b,area,position,total_reads,simpleClass,x_um,y_um
0,0,0,0,0,0,0,0,0,0,1,...,1,0,0,0,170.10,[1161.11337076 6178.3830543 ],870,67,1161.113371,6178.383054
1,1,0,0,1,1,0,0,0,0,0,...,9,1,0,1,206.55,[1162.91337076 6113.1330543 ],454,233,1162.913371,6113.133054
2,2,0,0,0,1,0,0,0,0,0,...,1,2,0,0,184.68,[1168.31337076 6046.0830543 ],269,97,1168.313371,6046.083054
3,3,0,0,0,0,2,0,0,0,0,...,3,0,0,0,87.48,[1163.81337076 6202.6830543 ],340,105,1163.813371,6202.683054
4,4,0,0,0,1,0,0,0,2,0,...,4,3,0,1,155.52,[1166.96337076 6083.8830543 ],241,72,1166.963371,6083.883054
5,5,0,0,0,2,1,0,0,1,0,...,11,3,0,0,233.28,[1171.46337076 6151.3830543 ],517,225,1171.463371,6151.383054
6,6,0,0,0,4,2,0,0,0,0,...,7,1,0,0,215.46,[1173.26337076 6206.7330543 ],429,193,1173.263371,6206.733054
7,7,0,1,0,2,1,0,0,0,0,...,1,0,0,0,204.12,[1184.06337076 6221.5830543 ],376,73,1184.063371,6221.583054
8,8,0,0,0,2,1,0,0,1,0,...,10,1,0,0,320.76,[1193.96337076 6186.4830543 ],789,227,1193.963371,6186.483054
9,9,0,0,0,0,0,0,0,1,0,...,2,3,0,0,194.40,[1197.11337076 6076.6830543 ],162,235,1197.113371,6076.683054


### Add the cells as points, double-plotting cells with high Rorb, Cux2 if available

In [25]:
cells = v.add_points(np.row_stack([[np.float(position.split(' ')[0].strip('[')), np.float(position.split(' ')[1].strip(']'))] for position in all_cells["position"]]), symbol='o', name = "all", opacity = 0.8, blending = "translucent")
colorlist = ["magenta", "green","yellow","cyan", "blue", "orange"]
for ii, gene_to_plot in enumerate(["Cux2", "Atp1a3" , "Vxn", "Pcp4" , "Fezf2" , "Rorb" , "Gm11549"]):

    if gene_to_plot in all_cells:
        cux2condition = all_cells[gene_to_plot]>0.5*all_cells[gene_to_plot].median()
        cux2cells = v.add_points(cells.data[cux2condition], symbol='disc',
                                 name = gene_to_plot+"+",edge_color= "black", size = 10,
                                 face_color= colorlist[ ii % len(colorlist)] , blending = "translucent", opacity = 0.3)


### annotate the layers in that viewer in a single `shapes` layer
#### Naming/order Convention:
0. outline of entire column as rectangle
1. Layer 1
1. Layer 2/3
2. Layer 4
3. Layer 5
4. Layer 6
5. single line segment, pointing from pia to white matter 

#### annotate the layer locations in a list

In [61]:
v.layers

[<Points layer 'all' at 0x1344a1350>,
 <Points layer 'Atp1a3+' at 0x131fcc310>,
 <Points layer 'Vxn+' at 0x13205a4d0>,
 <Points layer 'Pcp4+' at 0x132189bd0>,
 <Points layer 'Fezf2+' at 0x12141f850>,
 <Points layer 'Rorb+' at 0x132010290>,
 <Shapes layer 'Shapes' at 0x1330ff690>]

In [66]:
is_in_layer = {ii:[Polygon(p).intersects(Point(a))  for a  in v.layers[0].data] for ii,p in enumerate(v.layers[-1].data[:-2])}

In [ ]:
# ok:
# 1. make a couple functions to do this
#.  #. do a spatially partitioned search based on bounding box of polygon
# 2. throw annotation into separate vector matched to point cloud
#. 

In [32]:
v.layers[-1].data

[array([[3445.69646089, 5308.86210502],
        [2212.74395809, 7019.7031748 ],
        [1452.9011366 , 6278.97589599],
        [2284.42724314, 4888.32016609],
        [2284.42724314, 4888.32016609]]),
 array([[3369.23429017, 5318.41987636],
        [3307.10877646, 5528.69084583],
        [3230.64660575, 5495.23864614],
        [3302.32989079, 5304.08321935]]),
 array([[3201.97329173, 5256.29436266],
        [3125.51112101, 5514.35418882],
        [2929.57680855, 5882.3283854 ],
        [2771.87358145, 6173.84041125],
        [2480.3615556 , 6479.68909411],
        [2050.26184532, 6823.76886234],
        [1935.56858924, 6704.29672059],
        [2217.52284376, 6341.10140969],
        [2571.16038332, 5810.64510035],
        [2910.46126587, 5155.93776359]]),
 array([[1933.55200983, 6697.04759789],
        [2218.17354029, 6345.07288083],
        [2576.66630767, 5810.59275492],
        [2909.08687378, 5156.61503989],
        [2720.06341462, 5095.7799036 ],
        [2324.63502878, 5877.94594

In [28]:
annotations = v.layers[5]

In [27]:
v.layers

[<Points layer 'all' at 0x104ac0750>,
 <Points layer 'all [1]' at 0x12732a890>,
 <Points layer 'Cux2+' at 0x125db18d0>,
 <Points layer 'Fezf2+' at 0x127acadd0>,
 <Points layer 'Rorb+' at 0x127b4bad0>,
 <Shapes layer 'Shapes' at 0x125f6f310>]

In [68]:
v.add_points(all_cells[is_in_layer[1]][["x","y"]], symbol='o', size = 30)

<Points layer 'Points' at 0x133fe4390>

In [28]:
for ii,p in enumerate(annotations.data):
    print(p)

[[  322.69221525 10409.33143621]
 [10023.76221747  4247.2783876 ]
 [12605.16281892  8285.92126406]
 [ 3112.27028456 14864.32924839]
 [ 3112.27028456 14864.32924839]]
[[ 3195.5412717  14781.05826125]
 [  405.9632024  10492.60242336]
 [ 1696.66350312  9659.89255192]
 [ 2529.37337456 11533.48976265]
 [ 3445.35423313 13074.00302481]
 [ 4194.79311743 14073.25487053]
 [ 4194.79311743 14073.25487053]]
[[ 4278.06410457 13948.34838981]
 [ 3903.34466242 13573.62894767]
 [ 2820.82182956 12033.11568551]
 [ 2196.28942598 10784.05087836]
 [ 1655.02800955  9618.25705835]
 [ 4028.25114314  8161.01478334]
 [ 4611.14805314  9493.35057764]
 [ 5776.94187315 11491.85426908]
 [ 6526.38075744 12366.19963409]
 [ 6526.38075744 12366.19963409]]
[[ 6568.01625101 12366.19963409]
 [ 5860.21286029 11575.12525622]
 [ 4736.05453386  9659.89255192]
 [ 3944.98015599  8077.7437962 ]
 [ 5901.84835387  6911.94997619]
 [ 7941.98753888  9951.34100692]
 [ 8649.7909296  10992.22834622]
 [ 8649.7909296  10992.22834622]]
[[ 869

In [69]:
# depth coordinate calculation